# Comparison of Universities Curriculum

In [79]:
# Imports
# Data manipulation
import pandas as pd
# Set the display option to None to display all rows
pd.set_option('display.max_rows', None)

In [97]:
# The objective is to create simplify overview of the curriculum across 5 universities.
# Each curriculum is stored in separate excel table/sheet in ../data/curriculums.xlsx folder
# Each table has the same structure, with the following columns:
# - skola: university name
# - fakulta: faculty name
# - odbor: field of study
# - rocnik: year of study
# - semestr: semester
# - kod: unique identifier of the course use to outer join the curriculum across universities
# - predmet: course name
# The curriculums are stored in 5 sheets:
# - stu_fei
# - vut_fit
# - ukf_fai
# - utb_fai,
# _ vut_fsi
# with respective table names tb_{sheet_name}
# The goals is to create a overview that will have following structure:
# kod | stu_fei.predmet | vut_fit.predmet | ukf_fai.predmet | utb_fai.predmet | vut_fsi.predmet
# sorted by the kod, but each predment column will have only unique values
# Use the following steps
# 1. Load the data from each sheets to separate dataframes in for loop and use dictionary to store the dataframes
# 2. Merge the dataframes on the kod column using outer join
# 3. Simplify the dataframe, keep same columns with following conditions:
#  - each predmet column will have only unique values
#  - consider predemet columns to be completely idependent from each other to prevent duplicates
#  - partition the data by kod
#  - reframe storing list of unique values in the predmet columns
#  - Each predmet column should have consequitive list of unique predmet labels per university starting from first raw of partition by kod
#  - Each predmet label is sorted in ascending order in parition by kod and on separate row
#  - Number of raws for one partition equals to the max number of unique predmet lables from all universtities
# 4. Save the simplified dataframe in ../data/curriculums.csv to new sheet called merged_by_kod

# Step 1: Load the data
dfs = {}
universities = ['stu_fei', 'vut_fit', 'ukf_fai', 'utb_fai', 'vut_fsi']
for university in universities:
    dfs[university] = pd.read_excel('../data/curriculums.xlsx', sheet_name=university)

# Step 2: Merge the dataframes
df = dfs[universities[0]][['kod', 'predmet']].rename(columns={'predmet': f'{universities[0]}_predmet'})
for university in universities[1:]:
    df = pd.merge(
        df,
        dfs[university][['kod', 'predmet']].rename(columns={'predmet': f'{university}_predmet'}),  # Rename before merge
        on='kod',
        how='outer'
    )

# Step 3: Simplify the dataframe
df = df.groupby('kod').agg(lambda x: x.dropna().unique()).reset_index().sort_values('kod')
df_summary = pd.DataFrame()
for university in universities:
    df_subjects_per_university = pd.DataFrame()
    for i, code in enumerate(df['kod'].unique()):
        max_courses= max([len(df.loc[df['kod'] == code, f'{university}_predmet'].explode().unique()) for university in universities])
        # Create dataframe with kod and predment columns having unique and size equal to max_courses
        # If the number of unique courses is less than max_courses, fill the rest with NaN at the end
        df_subjects_per_code = df\
            .loc[df['kod'] == code, ['kod', f'{university}_predmet']]\
            .copy()\
            .explode(f'{university}_predmet')\
            .sort_values(f'{university}_predmet')

        # Create df_subjects_per_code_empty dataframe like df_subjects_per_code but with NaN values and size equal to max_courses - len(df_subjects_per_code) with kod value
        df_subjects_per_code_empty = pd.DataFrame(
            {
                'kod': [code] * (max_courses - len(df_subjects_per_code)),
                f'{university}_predmet': [None] * (max_courses - len(df_subjects_per_code))
            }
        )
        # add df_subjects_per_code_empty to df_subjects_per_code
        df_subjects_per_code = pd.concat([df_subjects_per_code, df_subjects_per_code_empty], ignore_index=True)
        df_subjects_per_university = pd.concat([df_subjects_per_university, df_subjects_per_code], ignore_index=True)
        # display all raws of df_subjects_per_university
    df_summary = pd.concat([df_summary, df_subjects_per_university[f'{university}_predmet']], axis=1) if not df_summary.empty else df_subjects_per_university

df_summary

with pd.ExcelWriter('../data/curriculums.xlsx', mode='a') as writer:
    df_summary.to_excel(writer, sheet_name='merged_by_kod')

,kod,stu_fei_volba,stu_fei_predmet,vut_fit_volba,vut_fit_predmet,ukf_fai_volba,ukf_fai_predmet,utb_fai_volba,utb_fai_predmet,vut_fsi_volba,vut_fsi_predmet
0,1,[P],"[Algoritmizacia a programovanie, Objektovo Ori...","[P, V, PVT]","[Úvod do softwarového inženýrství, Základy pro...","[P, PV]","[Programovanie a údajové štruktúry, Seminár z ...",[P],"[Programovací metody, Objektové programování, ...","[P, V]","[Úvod do programování a algoritmizace, Program..."
1,2,[P],"[Anlgicky Jazyk 1, Anglicky Jazyk 2]","[PVA, V]","[Zkouška z obecné angličtiny B1, Zkouška z ang...",[PV],"[Odborný jazyk: Anglický jazyk 1, Odborný jazy...",[P],"[Angličtina 1, Angličtina 2, Angličtina 3, Ang...",[P],"[General English I, General English Exam, Gene..."
2,3,[P],[Logicke Systemy],"[P, PVT]","[Návrh číslicových systémů, Základy logiky pro...",[P],[Logické systémy počítačov],[],[],[],[]
3,4,[P],"[Matematika 1, Matematika 2, Matematika 3]","[P, V]","[Diskrétní matematika, Lineární algebra, Matem...",[P],"[Diskrétna matematika 1, Diskrétna matematika ...",[P],"[Matematický seminář, Softwarová podpora inžen...","[P, V]","[Matematika I, Vybrané kapitoly z matematiky, ..."
4,5,[P],"[Telesna kultura, Telesna kultura 3, Telsna ku...",[],[],[],[],[P],"[Sportovní aktivity 1, Sportovní aktivity 2, S...",[],[]
5,6,[P],[Uvod do inzinierstva a bezpecnost v elektrote...,"[P, V]","[Elektronika pro informační technologie, Elekt...",[],[],[P],"[Elektrotechnika, Analogová a číslicová technika]","[P, PV]","[Základy elektrotechniky, Řídicí elektronika, ..."
6,7,[V],[Rozhranie clovek-stroj],[],[],[],[],[],[],[],[]
7,8,[V],[Zaklady kibernetiky],[],[],[],[],[],[],[],[]
8,9,[P],"[Uvod do fyziky, Fyzika]",[V],"[Fyzika 1, Fyzika v elektrotechnice, Fyzikální...",[],[],[P],"[Fyzikální seminář, Tepelné procesy, Mechanika...","[P, V, PV]","[Fyzika I, Vybrané kapitoly z fyziky I, Fyzika..."
9,10,[P],[Archtektura pocitacov],[],[],"[P, PV]","[Architektúra počítačov, Vybrané kapitoly z ha...",[P],[Hardware a operační systémy],[],[]
